In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
from xgboost import XGBRFRegressor
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
# Load the data
india_train = pd.read_csv("India_train.csv")
india_test = pd.read_csv("India_test.csv")
usa_train = pd.read_csv("USA_train.csv")
usa_test = pd.read_csv("USA_test.csv")
soil_rainfall = pd.read_csv("India_soil_rainfall_30years.csv")
commodity = pd.read_csv("USA_Commodity_prices.csv")
submission = pd.read_csv("submission.csv")

In [ ]:
india_train

In [ ]:
india_train.shape

In [ ]:
india_test.shape

In [ ]:
india_train.info()

In [ ]:
india_test.info()

In [ ]:
india_train.describe()

In [ ]:
india_test.describe()

In [ ]:
india_train.isnull().sum()

In [ ]:
india_train=india_train.fillna("0")

In [ ]:
india_train= india_train.astype(float, errors='ignore')

In [ ]:
india_train.dtypes

In [ ]:
india_train = india_train.drop(['State Name','Dist Name'], axis=1)

In [ ]:
fig = px.scatter(india_train, x="COTTON PRODUCTION (1000 tons)", y="COTTON YIELD (Kg per ha)")
fig.show()

In [ ]:
fig = px.scatter(india_train, x="COTTON AREA (1000 ha)", y="COTTON PRODUCTION (1000 tons)")
fig.show()

In [ ]:
# Load the training data
# Separate the features and the target variable for prediction
columns = india_train.columns

In [ ]:
columns

In [ ]:
columns = [col for col in columns if col not in ['COTTON YIELD (Kg per ha)']]

In [ ]:
#for demo perpose we are using number values

In [ ]:
features = columns
target = 'COTTON YIELD (Kg per ha)'

X = india_train[features]
y = india_train[target]

In [ ]:
X

In [ ]:
# Split the data into training set and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Define the model
model_fare = XGBRFRegressor()

# Train the model
model_fare.fit(X_train, y_train)

# Predict the validation set results
y_pred_val = model_fare.predict(X_val)

In [ ]:
# Calculate the MSE
mse = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"Validation MSE: {mse}")

In [ ]:
# Predict the yield in the test set
india_test['COTTON YIELD (Kg per ha)'] = model_fare.predict(india_test[features])

In [ ]:
submission_india=india_test['COTTON YIELD (Kg per ha)']

In [ ]:
submission_india

# Let train USA data

In [ ]:
columns = usa_train.columns

In [ ]:
columns

In [ ]:
usa_train=usa_train.fillna("0")

In [ ]:
usa_train= usa_train.astype(float, errors='ignore')

In [ ]:
usa_train.dtypes

In [ ]:
usa_train=usa_train.drop(['State','Planted (1000 Acres)',"Harvested (1000 Acres)"], axis=1)

In [ ]:
usa_train

In [ ]:
columns=usa_train.columns

In [ ]:
columns

In [ ]:
columns = [col for col in columns if col not in ['Yield (Pounds/ Harvested Area)']]

In [ ]:
features = columns
target = 'Yield (Pounds/ Harvested Area)'

X = usa_train[features]
y = usa_train[target]

In [ ]:
# Split the data into training set and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Define the model
model_fare = XGBRFRegressor()

# Train the model
model_fare.fit(X_train, y_train)

# Predict the validation set results
y_pred_val = model_fare.predict(X_val)

In [ ]:
# Calculate the MSE
mse = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"Validation MSE: {mse}")

In [ ]:
usa_test

In [ ]:
# Predict the yield in the test set
usa_test['Yield (Pounds/ Harvested Area)'] = model_fare.predict(usa_test[features])

In [ ]:
usa_test

In [ ]:
submission_usa=usa_test['Yield (Pounds/ Harvested Area)']

In [ ]:
submission_usa

In [ ]:
commodity

### We need to find relation and fill the data from year 2003 to 2022 with the USA data

In [ ]:
# Convert the 'Year' column to numeric type if it's not already
commodity['Year'] = pd.to_numeric(commodity['Year'])

# Select the rows between the years 2003 and 2022
submission_commodity = commodity[(commodity['Year'] >= 2003) & (commodity['Year'] <= 2022)]


In [ ]:
submission_commodity=submission_commodity["Cotton_Price[Dollar/ton]"]

### To create the final submission file

In [ ]:
submission = pd.concat([submission_india,submission_usa,submission_commodity],axis=1)

In [ ]:
submission

In [ ]:
submission = submission.fillna(0)

In [ ]:
submission

In [ ]:
submission.to_csv("submission.csv",index=False)